# ショット切り出し  

以下の処理を行います。  

* ショット切り出し
* 中断区間のデータ除外
* 物理変換 + 校正
* SPM計算
* 事象記録のタグ付け
* SPMから算出される、ショット当たりの最大サンプル数を超えたショット除外
* Elasticsearchインデックスへの保存
  * shots-yyyyMMddHHMMSS-data：切り出されたショットデータ
  * shots-yyyyMMddHHMMSS-meta：ショットのメタデータ
  

In [7]:
# ライブラリインポート
import os, sys
sys.path.append(os.path.join(os.getcwd(), '../src/'))
from cut_out_shot.cut_out_shot import CutOutShot
from utils import common
import logging

In [8]:
# ログ設定
LOG_FILE = os.path.join(
    common.get_config_value(common.APP_CONFIG_PATH, "log_dir"), "cut_out_shot/cut_out_shot.log"
)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.handlers.RotatingFileHandler(
            LOG_FILE, maxBytes=common.MAX_LOG_SIZE, backupCount=common.BACKUP_COUNT
        ),
        logging.StreamHandler(),
    ],
)

### CutOutShot クラスパラメーター一覧  

**【必須】**  
***displacement_func***  
変位センサーの物理変換式  

***load01_func***  
荷重センサー01の物理変換式  

***load02_func***  
荷重センサー02の物理変換式  

***load03_func***  
荷重センサー03の物理変換式  

***load04_func***  
荷重センサー04の物理変換式  


**【任意】**  
***min_spm***  
最小SPM。この値をもとに、1ショットに含まれる最大サンプル数を計算し、最大サンプル数を超えたショットは切り出し対象外とする。  
指定しない場合、既定値の15となる。  

***back_seconds_for_tagging***  
データ収集マネージャーで記録した事象について、何秒遡ってタグ付けするかを指定。指定しない場合、既定値の120となる。

***previous_size***  
ショットを検知したサンプルから何サンプル遡ってショットに含めるかを指定。これは切り出しショットに荷重開始点が確実に含まれるようにするため。指定しない場合、既定値の1000となる。


In [9]:
# 物理変換式設定
#displacement_func = lambda v: 135.0 - (v - 2.0) * 70.0 / 8.0
displacement_func = lambda v: v
load01_func = lambda v: v * 100000 / 38000.0
load02_func = lambda v: v * 100000 / 38000.0
load03_func = lambda v: v * 100000 / 38000.0
load04_func = lambda v: v * 100000 / 38000.0

In [10]:
cut_out_shot = CutOutShot(
    displacement_func=displacement_func,
    load01_func=load01_func,
    load02_func=load02_func,
    load03_func=load03_func,
    load04_func=load04_func
)

### cut_out_shot 関数パラメーター一覧
**【必須】**  
***rawdata_dir_name***  
ショット切り出しの対象とする生データファイルのディレクトリ（.pklファイルが含まれるディレクトリ）。  
ディレクトリ名はデータ収集開始日時で、yyyyMMddHHMMSSの形式となっている。

***start_displacement***  
ショット切り出しの開始を検知するための変位値。  

***end_displacement***  
ショット切り出しの終了を検知するための変位値。  

In [ ]:
# 適宜変更してください
rawdata_dir_name="20201201010000"
start_displacement=47.0
end_displacement=34.0

cut_out_shot.cut_out_shot(
    rawdata_dir_name=rawdata_dir_name, 
    start_displacement=start_displacement, 
    end_displacement=end_displacement
)

2021-02-05 10:26:37,248 [INFO] delete index 'shots-20201201010000-data' finished. result: {'acknowledged': True}
2021-02-05 10:26:37,452 [INFO] create index 'shots-20201201010000-data' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'shots-20201201010000-data'}
2021-02-05 10:26:37,516 [INFO] delete index 'shots-20201201010000-meta' finished. result: {'acknowledged': True}
2021-02-05 10:26:37,714 [INFO] create index 'shots-20201201010000-meta' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'shots-20201201010000-meta'}
2021-02-05 10:27:06,514 [INFO] 735312 shots detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20201201010000/tmp000.pkl.
2021-02-05 10:27:31,987 [INFO] Thoughput: 184263 doc/sec, processed_count: 10000000
2021-02-05 10:27:38,215 [INFO] 436180 shots detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20201201010000/tmp001.pkl.
2021-02-05 10:28:01,994 [INFO] Thoughput: 237311 doc/s

In [6]:
# 参考
# サンプルデータ 20201201010000
displacement_func = lambda v: v
load01_func = lambda v: v
load02_func = lambda v: v
load03_func = lambda v: v
load04_func = lambda v: v

cut_out_shot = CutOutShot(
    displacement_func=displacement_func,
    load01_func=load01_func,
    load02_func=load02_func,
    load03_func=load03_func,
    load04_func=load04_func,
    previous_size=1000
)

rawdata_dir_name="20201201010000"
start_displacement=47.0
end_displacement=34.0

cut_out_shot.cut_out_shot(
    rawdata_dir_name=rawdata_dir_name, 
    start_displacement=start_displacement, 
    end_displacement=end_displacement
)

2021-02-04 16:50:31,763 [INFO] delete index 'shots-20201201010000-data' finished. result: {'acknowledged': True}
2021-02-04 16:50:31,979 [INFO] create index 'shots-20201201010000-data' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'shots-20201201010000-data'}
2021-02-04 16:50:32,058 [INFO] delete index 'shots-20201201010000-meta' finished. result: {'acknowledged': True}
2021-02-04 16:50:32,259 [INFO] create index 'shots-20201201010000-meta' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'shots-20201201010000-meta'}
2021-02-04 16:50:55,919 [INFO] 367656 shots detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20201201010000/tmp000.pkl.
2021-02-04 16:51:18,769 [INFO] Thoughput: 215025 doc/sec, processed_count: 10000000
2021-02-04 16:51:25,101 [INFO] 436180 shots detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20201201010000/tmp001.pkl.
2021-02-04 16:51:48,397 [INFO] Thoughput: 262695 doc/s